In [1]:
import numpy as np
from numpy import random
import random
import matplotlib.pyplot as plt
from scipy.signal import cwt,ricker
from scipy import stats
from sklearn.preprocessing import StandardScaler,normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
import os
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.optimizers
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from tensorflow.keras.layers import Flatten,concatenate,Layer,Dense,LSTM,Activation,MaxPooling2D,Dropout,Conv2D,BatchNormalization,Reshape,UpSampling2D,ZeroPadding2D
import radiomics
from radiomics.featureextractor import RadiomicsFeatureExtractor
import SimpleITK as sitk
import pandas as pd
import tensorflow_addons as tfa

2022-10-24 05:14:51.162399: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
image = sitk.ReadImage('brainClassify/Training/glioma_tumor/gg (1).jpg')
ma = sitk.GetArrayFromImage(image)
print(ma.shape)

(512, 512, 3)


In [3]:
#jpg rgb scale no point
# for i in ma:
#     for j in i:
#         print(j)

In [4]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [5]:
print(rgb2gray(ma).shape)

(512, 512)


In [6]:
rgb2gray(ma).reshape(512,512,1).shape

(512, 512, 1)

In [7]:
dict = './brainClassify/Training/'

# Function takes a list of folders, extract features from it, change the extractor inside function and the return dictionary
# to run exactly which feature, the key of the return dictionary can be found in mask.ipy and also listed in pyradiomics library
def LoadDataFromImages(foldernames):
    input_array = []
    input_array_greyscale =[]
    #output_array=[]
    o1=[]
    o2=[]
    o3=[]
    o4=[]
    o5=[]
    o6=[]
    extractor = RadiomicsFeatureExtractor()
    # Disable all classes except firstorder
    extractor.disableAllFeatures()
    extractor.enableFeaturesByName(firstorder=['Skewness']) 
    extractor.enableFeaturesByName(glrlm=['GrayLevelVariance'])

    extractor.enableFeaturesByName(glcm=['Autocorrelation','ClusterShade','DifferenceEntropy'])  # change here to extract
    extractor.enableFeaturesByName(glszm=['SizeZoneNonUniformity'])

    for i in foldernames:
        files  = sorted(os.listdir(dict + i))
        for j in files:

            im = sitk.ReadImage(dict + i +'/' + j)
            # need resize to 512 512 for all
            
            im = sitk.Resample(im, [512, 512])
            #resampled_img_arr = sitk.GetArrayFromImage(resampled_img)
            
            # reshape and fill the input array
            ori_im = sitk.GetArrayFromImage(im)
            input_array.append(ori_im)
            #print(resampled_img_arr.shape)
            
            greyscale_im = rgb2gray(ori_im).reshape(512,512,1)
            
            input_array_greyscale.append(greyscale_im)
            greyscale_im_get =  sitk.GetImageFromArray(greyscale_im)
            ma_arr = np.ones(greyscale_im_get.GetSize()[::-1])  # reverse the order as image is xyz, array is zyx
            ma = sitk.GetImageFromArray(ma_arr)
            ma.CopyInformation(greyscale_im_get)  # Copy geometric info
            
            # extract features and fill the output array
            features = extractor.execute(greyscale_im_get, ma)
            
            o1.append(features['original_glcm_ClusterShade'])  # change here to extract
            o2.append(features['original_firstorder_Skewness'])
            o3.append(features['original_glrlm_GrayLevelVariance'])
            o4.append(features['original_glcm_Autocorrelation'])
            o5.append(features['original_glcm_DifferenceEntropy'])
            o6.append(features['original_glszm_SizeZoneNonUniformity'])

            
    return input_array,input_array_greyscale, o1,o2,o3,o4,o5,o6

In [41]:
inn ,inn_grey, o1,o2,o3,o4,o5,o6 = LoadDataFromImages(['glioma_tumor',
                                                        'meningioma_tumor'
    ,'pituitary_tumor', 'no_tumor'
                                                      ])

In [42]:
inn = np.array(inn)
inn_grey = np.array(inn_grey)
o1 = np.array(o1)
o2 = np.array(o2)
o3 = np.array(o3)
o4 = np.array(o4)
o5 = np.array(o5)
o6 = np.array(o6)

np.save("brainclassin.npy",inn)
np.save("brainclassingrey.npy",inn_grey)
np.save("brainclass1.npy",o1)
np.save("brainclass2.npy",o2)
np.save("brainclass3.npy",o3)
np.save("brainclass4.npy",o4)
np.save("brainclass5.npy",o5)
np.save("brainclass6.npy",o6)

In [16]:
inn_grey.shape

(2870, 512, 512, 1)

In [2]:
# extracting
class_names = ['glioma_tumor', 'meningioma_tumor','pituitary_tumor', 'no_tumor']
dict = './brainClassify/Training/'


def load_labels():
    #trainingdata = []
    labels = []
    for i in class_names:
        path = os.path.join(dict, i)
        label_num = class_names.index(i)
        for j in sorted(os.listdir(path)):
            temp = [0,0,0,0]
            temp[label_num] = 1
            #img = sitk.ReadImage(dict + i +'/' + j)
            #trainingdata.append([sitk.GetArrayFromImage(img).reshape(512,512,1),label_num])
            labels.append(temp)
            
            
    return labels
    
            

In [3]:
labels = load_labels()

In [4]:
labels =np.array(labels)
labels.shape

(2870, 4)

In [5]:
inn = np.load('brainclassin.npy')
inn_grey =  np.load('brainclassingrey.npy')
o1 =  np.load("brainclass1.npy")
o2 =  np.load("brainclass2.npy")
o3 =  np.load("brainclass3.npy")
o4 =  np.load("brainclass4.npy")
o5 =  np.load("brainclass5.npy")
o6 =  np.load("brainclass6.npy")

In [6]:
from sklearn.model_selection import train_test_split

In [7]:

# change the random_state to get different train and val
def split_training_val(input_group):

    i_train, i_val = train_test_split(input_group, test_size=0.1,random_state= 60, shuffle=True)

    return np.array(i_train), np.array(i_val)

inn, inn_val = split_training_val(inn)
inn_grey, inn_grey_val = split_training_val(inn_grey)
o1, o1_val = split_training_val(o1)
o2, o2_val = split_training_val(o2)
o3, o3_val = split_training_val(o3)
o4, o4_val = split_training_val(o4)
o5, o5_val = split_training_val(o5)
o6, o6_val = split_training_val(o6)

In [8]:
labels, labels_val =  split_training_val(labels)

In [9]:
labels.shape

(2583, 4)

In [10]:
inn_grey.shape

(2583, 512, 512, 1)

In [11]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [12]:
# Load test group
dict = './brainClassify/Testing/'
class_names = ['glioma_tumor', 'meningioma_tumor','pituitary_tumor', 'no_tumor']
# Function takes a list of folders, extract features from it, change the extractor inside function and the return dictionary
# to run exactly which feature, the key of the return dictionary can be found in mask.ipy and also listed in pyradiomics library
def LoadTestGroup():
    input_array = []
    input_array_greyscale =[]
    labels = []

    for i in class_names:
        path = os.path.join(dict, i)
        label_num = class_names.index(i)
        files  = sorted(os.listdir(path))
        for j in files:
            im = sitk.ReadImage(dict + i +'/' + j)
            im = sitk.Resample(im, [512, 512])

            ori_im = sitk.GetArrayFromImage(im)
            input_array.append(ori_im)
            #print(resampled_img_arr.shape)
            
            greyscale_im = rgb2gray(ori_im).reshape(512,512,1)
            
            input_array_greyscale.append(greyscale_im)
            temp = [0,0,0,0]
            temp[label_num] = 1
            labels.append(temp)
            
    return input_array,input_array_greyscale, labels

In [13]:
test_data, test_datagrey, test_labels = LoadTestGroup()

In [14]:
test_data=np.array(test_data)
test_datagrey=np.array(test_datagrey)
test_labels=np.array(test_labels)
test_data.shape

(394, 512, 512, 3)

In [15]:


def MakeModel(modelname):
    input_layer = keras.Input(shape=(512,512,1))
    x = Conv2D(64,(3,3),strides=(1, 1), activation='relu')(input_layer)
    pool1 = MaxPooling2D(pool_size=(2, 2))(x)
    x1 = Conv2D(64,(3,3),strides=(1, 1), activation='relu')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(x1)


    flat = Flatten()(pool2)
    d1 = Dense(147, activation='relu',kernel_initializer = tf.keras.initializers.GlorotNormal())(flat)

    out = Dense(4, activation = 'softmax', name= modelname)(d1)

    model = keras.Model(input_layer, out, name= modelname)
    
    return model


In [16]:
model = MakeModel("CNN")

2022-10-24 05:15:18.659953: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-24 05:15:19.416309: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30941 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:3d:00.0, compute capability: 7.0


In [17]:
model.summary()

Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512, 512, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 510, 510, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 255, 255, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 253, 253, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 126, 126, 64)     0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1016064)           0       

In [18]:
mm_save = ModelCheckpoint('grey.hdf5', save_best_only=True, monitor='val_loss', mode='min')
es = tf.keras.callbacks.EarlyStopping(patience=2)

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(
     #learning_rate= 0.01
),
              loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=['accuracy', tf.keras.metrics.AUC(),
tfa.metrics.F1Score(num_classes=4, threshold=0.5)]
                   )



In [20]:
model.fit(inn_grey, labels, 
               validation_data=([inn_grey_val,labels_val]), 
                batch_size=64,
                epochs=20,
          callbacks=[es,
                   #  mm_save
                    ],
                          shuffle = True)

Epoch 1/20


2022-10-24 05:15:24.466523: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


41/41 [==============================] - 15s 258ms/step - loss: 569.7259 - accuracy: 0.6252 - auc: 0.7697 - f1_score: 0.5818 - val_loss: 0.5907 - val_accuracy: 0.7909 - val_auc: 0.9495 - val_f1_score: 0.7747
Epoch 2/20
41/41 [==============================] - 9s 219ms/step - loss: 0.2939 - accuracy: 0.8897 - auc: 0.9848 - f1_score: 0.8881 - val_loss: 0.5402 - val_accuracy: 0.8293 - val_auc: 0.9627 - val_f1_score: 0.8184
Epoch 3/20
41/41 [==============================] - 9s 220ms/step - loss: 0.0930 - accuracy: 0.9698 - auc: 0.9981 - f1_score: 0.9691 - val_loss: 0.6239 - val_accuracy: 0.8502 - val_auc: 0.9632 - val_f1_score: 0.8417
Epoch 4/20
41/41 [==============================] - 9s 220ms/step - loss: 0.0254 - accuracy: 0.9934 - auc: 0.9999 - f1_score: 0.9935 - val_loss: 0.5939 - val_accuracy: 0.8885 - val_auc: 0.9677 - val_f1_score: 0.8812


In [21]:
model.evaluate(test_datagrey,test_labels)

13/13 [==============================] - 1s 54ms/step - loss: 6.9813 - accuracy: 0.6802 - auc: 0.8139 - f1_score: 0.6409


[6.981349468231201,
 0.6802030205726624,
 0.8138910531997681,
 array([0.30508474, 0.8200837 , 0.6962963 , 0.74204946], dtype=float32)]

RGB based

In [22]:
def MakeModel(modelname):
    input_layer = keras.Input(shape=(512,512,3))
    x = Conv2D(64,(3,3), strides=(1, 1),activation='relu')(input_layer)
    pool1 = MaxPooling2D(pool_size=(2, 2))(x)
    x1 = Conv2D(64,(3,3),strides=(1, 1), activation='relu')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(x1)


    flat = Flatten()(pool2)
    d1 = Dense(147, activation='relu',kernel_initializer = tf.keras.initializers.GlorotNormal())(flat)

    out = Dense(4, activation = 'softmax', name= modelname)(d1)

    model = keras.Model(input_layer, out, name= modelname)
    
    return model


In [23]:
model = MakeModel("CNNRGB")

In [24]:
mm_save = ModelCheckpoint('rgbbased.hdf5', save_best_only=True, monitor='val_loss', mode='min')
es = tf.keras.callbacks.EarlyStopping(patience=2)

In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(
     #learning_rate= 0.01
),
              loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=['accuracy', tf.keras.metrics.AUC(),
tfa.metrics.F1Score(num_classes=4, threshold=0.5)]
                   )


In [26]:
model.fit(inn, labels, 
               validation_data=([inn_val,labels_val]), 
                batch_size=64,
                epochs=20,
          callbacks=[es,mm_save],
                          shuffle = True)

Epoch 1/20
41/41 [==============================] - 12s 280ms/step - loss: 297.4590 - accuracy: 0.6643 - auc_1: 0.7950 - f1_score: 0.6402 - val_loss: 0.6240 - val_accuracy: 0.8362 - val_auc_1: 0.9474 - val_f1_score: 0.7940
Epoch 2/20
41/41 [==============================] - 9s 217ms/step - loss: 0.2957 - accuracy: 0.9032 - auc_1: 0.9848 - f1_score: 0.8975 - val_loss: 0.6322 - val_accuracy: 0.8746 - val_auc_1: 0.9628 - val_f1_score: 0.8630
Epoch 3/20
41/41 [==============================] - 9s 217ms/step - loss: 0.0815 - accuracy: 0.9710 - auc_1: 0.9981 - f1_score: 0.9694 - val_loss: 0.8131 - val_accuracy: 0.8676 - val_auc_1: 0.9636 - val_f1_score: 0.8648


In [27]:
model.evaluate(test_data,test_labels)

13/13 [==============================] - 1s 46ms/step - loss: 6.3224 - accuracy: 0.6599 - auc_1: 0.7940 - f1_score: 0.6057


[6.322449684143066,
 0.6598984599113464,
 0.7940168380737305,
 array([0.3089431 , 0.7816092 , 0.57142854, 0.7608695 ], dtype=float32)]

# FINs section

In [37]:
inp = np.load('brainclassingrey.npy')
oup = np.load('brainclass6.npy')

# base_model =  load_model("SizeZoneNonUniformity2.h5")

# base_model.summary()

In [36]:
# how many layers for saving
newlayers = base_model.layers[3].output
n1 = Dense(64, activation='relu',name = '3')(newlayers)
out = Dense(1, activation='linear',name='4')(n1)
new_model = keras.Model(base_model.input, outputs = out)

In [37]:
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 512, 512, 1)]     0         
                                                                 
 flatten_19 (Flatten)        (None, 262144)            0         
                                                                 
 dense_54 (Dense)            (None, 64)                16777280  
                                                                 
 dense_55 (Dense)            (None, 64)                4160      
                                                                 
 3 (Dense)                   (None, 64)                4160      
                                                                 
 4 (Dense)                   (None, 1)                 65        
                                                                 
Total params: 16,785,665
Trainable params: 16,785,665
Non-t

In [38]:
for i in range(len(base_model.layers)): 
    keras.layers.trainable = True   # True--> fine tine, False-->frozen

In [15]:
inp = np.load('brainclassingrey.npy')
oup = np.load('brainclass6.npy')

In [16]:
new_model = load_model('BC_SizeZoneNonUniformity.h5')

2022-10-24 04:10:04.634064: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-24 04:10:05.302505: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30941 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:3d:00.0, compute capability: 7.0


In [17]:

new_model.compile(optimizer=tf.keras.optimizers.Adam(
 learning_rate= 0.001
), loss="mse")

In [18]:

new_model.fit(inp, oup, 
              epochs=20, 
          batch_size=64,
                shuffle=True
            )

Epoch 1/20
45/45 [==============================] - 2s 18ms/step - loss: 11349784.0000
Epoch 2/20
45/45 [==============================] - 1s 18ms/step - loss: 132634.8281
Epoch 3/20
45/45 [==============================] - 1s 18ms/step - loss: 49404.9961
Epoch 4/20
45/45 [==============================] - 1s 17ms/step - loss: 40402.0859
Epoch 5/20
45/45 [==============================] - 1s 17ms/step - loss: 37911.9453
Epoch 6/20
45/45 [==============================] - 1s 19ms/step - loss: 38302.9414
Epoch 7/20
45/45 [==============================] - 1s 17ms/step - loss: 35825.7734
Epoch 8/20
45/45 [==============================] - 1s 17ms/step - loss: 33390.3867
Epoch 9/20
45/45 [==============================] - 1s 18ms/step - loss: 31461.0254
Epoch 10/20
45/45 [==============================] - 1s 17ms/step - loss: 30646.8047
Epoch 11/20
45/45 [==============================] - 1s 17ms/step - loss: 31772.5430
Epoch 12/20
45/45 [==============================] - 1s 18ms/step - lo

In [19]:
new_model.save("./BC_SizeZoneNonUniformity.h5")

In [35]:
# loading FINs

# load the 6 Fins to attach
m1 = load_model('BC_ClusterShade.h5')
m2 = load_model('BC_Skewness.h5')
m3 = load_model('BC_GLV.h5')
m4 = load_model('BC_Autocorrelation.h5')
m5 = load_model('BC_DifferenceEntropy.h5')
m6 = load_model('BC_SizeZoneNonUniformity.h5')

m1._name = 'feature1'
m2._name = 'feature2'
m3._name = 'feature3'
m4._name = 'feature4'
m5._name = 'feature5'
m6._name = 'feature6'

def MakeModel(modelname):
    input_layer = keras.Input(shape=(512,512,1))
    
    l1 = m1(input_layer)
    l2 = m2(input_layer)
    l3 = m3(input_layer)
    l4 = m4(input_layer)
    l5 = m5(input_layer)
    l6 = m6(input_layer)
    
    # Attacted a CNN portion with the FINs
    
    x = Conv2D(64,(3,3), strides=(1, 1),activation='relu')(input_layer)
    pool1 = MaxPooling2D(pool_size=(2, 2))(x)
    x1 = Conv2D(64,(3,3),strides=(1, 1), activation='relu')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(x1)
    
    flat = Flatten()(pool2)
    
    n_d = concatenate([l1
                       ,l2
                       ,l3
                      ,l4
                       ,l5, l6
                        , flat
                      ])

    d1 = Dense(32, activation='relu',kernel_initializer = tf.keras.initializers.GlorotNormal()
         ,name='dense_N_1')(n_d)
    
    d2 = Dense(32, activation='relu',kernel_initializer = tf.keras.initializers.GlorotNormal()
     ,name='dense_N_2')(d1)
    d2 = Dense(32, activation='relu',kernel_initializer = tf.keras.initializers.GlorotNormal()
     ,name='dense_N_3')(d2)
    

    out = Dense(4, activation = 'softmax', name= modelname)(d2)

    model = keras.Model(input_layer, out, name= modelname)
    
    return model

In [36]:
FinModel = MakeModel('Fin')

In [37]:
FinModel.summary()

Model: "Fin"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 510, 510, 64  640         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_6 (MaxPooling2D)  (None, 255, 255, 64  0          ['conv2d_6[0][0]']               
                                )                                                               

In [38]:
#mm_save = ModelCheckpoint('finBTC.hdf5', save_best_only=True, monitor='val_loss', mode='min')
es = tf.keras.callbacks.EarlyStopping(patience=2)

In [39]:
FinModel.compile(optimizer=tf.keras.optimizers.Adam(
     #learning_rate= 0.01
),
              loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=['accuracy',tf.keras.metrics.AUC(),
tfa.metrics.F1Score(num_classes=4, threshold=0.5)]
                   )

In [40]:
FinModel.fit(inn_grey, labels, 
               validation_data=([inn_grey_val,labels_val]), 
                batch_size=64,
                epochs=20,
             callbacks=[es,
                        #mm_save
                       ],
                          shuffle = True)

Epoch 1/20
41/41 [==============================] - 12s 246ms/step - loss: 29.0524 - accuracy: 0.5257 - auc_3: 0.7057 - f1_score: 0.4600 - val_loss: 0.7128 - val_accuracy: 0.7631 - val_auc_3: 0.9135 - val_f1_score: 0.7312
Epoch 2/20
41/41 [==============================] - 9s 227ms/step - loss: 0.4078 - accuracy: 0.8719 - auc_3: 0.9718 - f1_score: 0.8636 - val_loss: 0.7298 - val_accuracy: 0.8293 - val_auc_3: 0.9427 - val_f1_score: 0.8200
Epoch 3/20
41/41 [==============================] - 9s 227ms/step - loss: 0.2400 - accuracy: 0.9392 - auc_3: 0.9907 - f1_score: 0.9367 - val_loss: 0.5200 - val_accuracy: 0.8746 - val_auc_3: 0.9658 - val_f1_score: 0.8674
Epoch 4/20
41/41 [==============================] - 9s 227ms/step - loss: 0.1079 - accuracy: 0.9698 - auc_3: 0.9972 - f1_score: 0.9690 - val_loss: 0.7902 - val_accuracy: 0.8432 - val_auc_3: 0.9504 - val_f1_score: 0.8351
Epoch 5/20
41/41 [==============================] - 9s 229ms/step - loss: 0.0495 - accuracy: 0.9841 - auc_3: 0.9992 - 

In [42]:
FinModel.evaluate(test_datagrey,test_labels)

13/13 [==============================] - 1s 45ms/step - loss: 4.7472 - accuracy: 0.6980 - auc_3: 0.8268 - f1_score: 0.6416


[4.747223377227783,
 0.6979695558547974,
 0.8268121480941772,
 array([0.2905983, 0.8015564, 0.6993007, 0.7749077], dtype=float32)]